In [1]:
# import time
# import datetime
import numpy as np
# import math
# from sklearn.linear_model import LogisticRegression
import pandas as pd
import os

In [2]:
train_user = pd.read_csv("../tianchi_fresh_comp_train_user.csv")

In [3]:
train_user.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,10001082,285259775,1,97lk14c,4076,2014-12-08 18
1,10001082,4368907,1,NaN,5503,2014-12-12 12
2,10001082,4368907,1,NaN,5503,2014-12-12 12
3,10001082,53616768,1,NaN,9762,2014-12-02 15
4,10001082,151466952,1,NaN,5232,2014-12-12 11


In [4]:
train_item = pd.read_csv("../tianchi_fresh_comp_train_item.csv")

In [5]:
train_item.head()

,item_id,item_geohash,item_category
0,100002303,NaN,3368
1,100003592,NaN,7995
2,100006838,NaN,12630
3,100008089,NaN,7791
4,100012750,NaN,9614


In [6]:
train_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620918 entries, 0 to 620917
Data columns (total 3 columns):
item_id          620918 non-null int64
item_geohash     203410 non-null object
item_category    620918 non-null int64
dtypes: int64(2), object(1)
memory usage: 14.2+ MB


In [7]:
train_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23291027 entries, 0 to 23291026
Data columns (total 6 columns):
user_id          int64
item_id          int64
behavior_type    int64
user_geohash     object
item_category    int64
time             object
dtypes: int64(4), object(2)
memory usage: 1.0+ GB


In [12]:
train = pd.merge(train_item, train_user, on=['item_id', 'item_category'], how='left')

In [13]:
train.info(0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4279962 entries, 0 to 4279961
Data columns (total 7 columns):
item_id          int64
item_geohash     object
item_category    int64
user_id          int64
behavior_type    int64
user_geohash     object
time             object
dtypes: int64(4), object(3)
memory usage: 261.2+ MB


In [14]:
train.head()

,item_id,item_geohash,item_category,user_id,behavior_type,user_geohash,time
0,100002303,NaN,3368,20723532,1,NaN,2014-11-22 18
1,100002303,NaN,3368,20723532,1,NaN,2014-11-22 18
2,100002303,NaN,3368,20723532,1,NaN,2014-11-22 18
3,100002303,NaN,3368,44416664,1,94mn0i4,2014-12-16 15
4,100002303,NaN,3368,44416664,1,94mn0j6,2014-12-16 15


In [15]:
train['time_year'] = pd.DatetimeIndex(
    train['time']
).year
train['time_year'] = train.time_year.apply(
    lambda x: int(x) if pd.notnull(x) else "NAN"
)

In [16]:
train.head()

,item_id,item_geohash,item_category,user_id,behavior_type,user_geohash,time,time_year
0,100002303,NaN,3368,20723532,1,NaN,2014-11-22 18,2014
1,100002303,NaN,3368,20723532,1,NaN,2014-11-22 18,2014
2,100002303,NaN,3368,20723532,1,NaN,2014-11-22 18,2014
3,100002303,NaN,3368,44416664,1,94mn0i4,2014-12-16 15,2014
4,100002303,NaN,3368,44416664,1,94mn0j6,2014-12-16 15,2014


In [17]:
print(train.isnull().values.any())

True


In [18]:
columns_with_Nan = train.columns[train.isnull().any()].tolist()
print(columns_with_Nan)

['item_geohash', 'user_geohash']


In [20]:
train['time_month'] = pd.DatetimeIndex(train['time']).month
train['time_month'] = train.time_month.apply(lambda x: int(x))

In [21]:
train['time_day'] = pd.DatetimeIndex(train['time']).day
train['time_day'] = train.time_day.apply(lambda x: int(x))

In [24]:
train.head()

,item_id,item_geohash,item_category,user_id,behavior_type,user_geohash,time,time_year,time_month,time_day
0,100002303,NaN,3368,20723532,1,NaN,2014-11-22 18,2014,11,22
1,100002303,NaN,3368,20723532,1,NaN,2014-11-22 18,2014,11,22
2,100002303,NaN,3368,20723532,1,NaN,2014-11-22 18,2014,11,22
3,100002303,NaN,3368,44416664,1,94mn0i4,2014-12-16 15,2014,12,16
4,100002303,NaN,3368,44416664,1,94mn0j6,2014-12-16 15,2014,12,16


In [25]:
train = train.drop(['item_geohash', 'user_geohash'], axis=1)

In [26]:
train.head()

,item_id,item_category,user_id,behavior_type,time,time_year,time_month,time_day
0,100002303,3368,20723532,1,2014-11-22 18,2014,11,22
1,100002303,3368,20723532,1,2014-11-22 18,2014,11,22
2,100002303,3368,20723532,1,2014-11-22 18,2014,11,22
3,100002303,3368,44416664,1,2014-12-16 15,2014,12,16
4,100002303,3368,44416664,1,2014-12-16 15,2014,12,16


In [27]:
train = train.drop_duplicates(keep='first')

In [28]:
train.head()

,item_id,item_category,user_id,behavior_type,time,time_year,time_month,time_day
0,100002303,3368,20723532,1,2014-11-22 18,2014,11,22
3,100002303,3368,44416664,1,2014-12-16 15,2014,12,16
5,100002303,3368,142281897,1,2014-12-02 23,2014,12,2
6,100003592,7995,109084811,1,2014-11-24 18,2014,11,24
7,100006838,12630,134325108,1,2014-11-19 21,2014,11,19


In [40]:
# 最后三天可能购买，其余时间不够买
# 取最后三天的数据
train_last_3 = train[train['time_month'] == 12]

In [43]:
train_last_3 = train_last_3[train_last_3['time_day'] > 15]

In [44]:
train_last_3.head()

,item_id,item_category,user_id,behavior_type,time,time_year,time_month,time_day
3,100002303,3368,44416664,1,2014-12-16 15,2014,12,16
21,10003007,5827,135666003,1,2014-12-16 22,2014,12,16
66,100044374,10576,20758953,1,2014-12-18 14,2014,12,18
100,100044374,10576,23891314,1,2014-12-18 21,2014,12,18
127,100079162,4778,132236660,3,2014-12-17 20,2014,12,17


In [45]:
train_last_3.count()

item_id          104424
item_category    104424
user_id          104424
behavior_type    104424
time             104424
time_year        104424
time_month       104424
time_day         104424
dtype: int64

In [49]:
train_last_3 = train_last_3[['user_id', 'item_id', 'item_category', 'behavior_type']]
# train_score = train_last_3.groupby(['behavior_type']).sum()

In [54]:
# train_last_3 = train_last_3.reset_index()
train_last_3 = train_last_3.drop(['index'], axis=1)

In [55]:
train_last_3.head()

,user_id,item_id,item_category,behavior_type
0,44416664,100002303,3368,1
1,135666003,10003007,5827,1
2,20758953,100044374,10576,1
3,23891314,100044374,10576,1
4,132236660,100079162,4778,3


In [57]:
score_tmp = train_last_3.groupby(['user_id', 'item_id', 'item_category']).sum()

In [58]:
score_tmp.head()

behavior_type
user_id item_id   item_category               
59436   184081436 150                        1
        238861461 4582                       1
60723   202829025 8270                       1
        371933634 7939                       1
61797   83261906  3064                       1

In [59]:
score_tmp = score_tmp.groupby(['user_id', 'item_id']).sum()

In [107]:
score_tmp['behavior_type']

user_id    item_id  
59436      184081436    1
           238861461    1
60723      202829025    1
           371933634    1
61797      83261906     1
106362     38830684     1
           149517272    1
           200051509    1
           236862264    1
           308735138    1
           342598663    1
           364101379    1
           377408757    1
134211     6491625      1
           79679783     1
           96616269     1
           99999162     1
           291358470    1
           330807277    1
           374778111    1
137907     166251701    1
140106     4679677      1
           10163508     1
           14264945     3
           22283086     1
           38855121     3
           50545277     1
           53253613     1
           74839994     1
           77044060     2
                       ..
142411581  171751161    1
           184099792    1
           190257825    1
           214954648    1
           216739112    1
           222460581    1
           225039

In [112]:
np.where(score_tmp['behavior_type']

ValueError: Array conditional must be same shape as self

In [66]:
# score_tmp

In [67]:
# 用户三天之内进行了购买，那么三天后进行购买的几率会非常小
train_last_3['behavior_type'] = train_last_3['behavior_type'].apply(lambda x: int(x) if x!=4 else 0)

In [71]:
train_score = train_last_3.groupby(['user_id', 'item_id', 'item_category']).sum()

In [72]:
train_score.head()

behavior_type
user_id item_id   item_category               
59436   184081436 150                        1
        238861461 4582                       1
60723   202829025 8270                       1
        371933634 7939                       1
61797   83261906  3064                       1

In [88]:
train_score_1 = train_score['behavior_type'].values

In [89]:
train_score_

array([1, 1, 1, ..., 1, 2, 1], dtype=int64)

In [113]:
sub = pd.read_csv("../../WSDM-KKBox'sChurnPredictionChallenge/data/sample_submission_zero.csv")

In [114]:
sub.shape

(970960, 2)

In [115]:
sub.head()

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,0


In [121]:
sub['is_churn'][0:617460] =  1

E:\anacoda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [122]:
sub.head()

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1


In [139]:
sub.to_csv("G:\PythonProjects\KaggleOrOthersJourney/sub3.csv", index=False)

In [138]:
sub["is_churn"][617459]

1